# Duplicate detection

**Question:** What is the (most recent) population number of London?

Two datasets: 
* [UrbanAudit](./data/urb_cpop1.csv) - indicator DE1001V is the total population, names of cities are in a separate file [cities.csv](./data/cities.csv)
* [UNData](./data/UNdata_Export_20161031_132143657.csv) - total population is indicated by the values 

    "Total","Both Sexes"
    
  in the 3rd and 4th column, city name is in the 5th column, some city names are in upper case. The source year is in the 9th column and the actual population number is in the 10th column.
  
Let's try to get the data into a unique format into **one** dictionary. For the EU data we have done this last time already, let's take  the same code and slightly adapt it:

In [150]:
import csv # for handling csv/tsv files

urbanAuditFile="./data/urb_cpop1.tsv"
cityCodeFile="./data/cities.csv"

#Building the cityCode to Label map
cityCodeMap={}
with open(cityCodeFile) as f:
    csvfile = csv.reader(f)
    for i,row in enumerate(csvfile):
        cityCodeMap[row[3]]= row[1]
#cityCodeMap

recentCityPopulationEU={} # key= cityname-lowercase-ISO-20letter-country code, 
                          # value = {'year': year, 'pop':population}

## a quick trick to be able to convert the yearIndex to its label
## We know this snippet from last time... now we will use the lower-case for the labels, in order 
## to avoid confusion with upper and lower case names:

headerRow=[]
with open(urbanAuditFile) as f:
    csvfile = csv.reader(f, delimiter="\t")
    for i, row in enumerate(csvfile):
        if i==0:
            headerRow=row
        if 'DE1001V' in row[0] and len(row[0])> len("DE1001V,AT"): # same filter conditition as before
            for yearIndex in range (1,27): # this generates all numbers between 1 and 27
                popCount=row[yearIndex].strip().split(" ")[0]
                if popCount != ":" and int(popCount)>0: # check if value exists and we can convert it
                    #ok we have a population value >0, that is hte most recent year, lets store this
                    cityPopulation=int(popCount)
                    cityCode=row[0].split(",")[1]
                    cityLabel=cityCodeMap[cityCode].lower() # lets just merge the mapping from city code to label in this code
                    # small modification, we also add the 2 letter country code in uppercase to the key (we will need this later:
                    cityLabelCC = cityLabel + (row[0].split(',')[1][0:2])  
                    recentCityPopulationEU[cityLabelCC]={'year': headerRow[yearIndex],'pop': cityPopulation, } # we also convert the year index to the year label
                    break
                    
## Again, we could print this: 
# print("Number of cities: "+str(len(recentCityPopulationEU.keys())))
# for city, pop in recentCityPopulationEU.items():
#    print("  "+city+":"+str(pop['pop'])+" in "+str(pop['year']))

# Example:
print(recentCityPopulationEU['wienAT'])

{'year': '2013 ', 'pop': 1741246}


Now let's look into the UN Data file and while we build up a similar dictionary, try to cautiously check for duplicates. To this end, for the moment, we also store the original rows in the dictionary:

In [133]:
undatafile = 'data/UNdata_Export_20161031_132143657.csv'

recentCityPopulationUN={}

with open(undatafile) as f:
    csvfile = csv.reader(f)

    for row in csvfile:
        if len(row) > 10 and row[2] == "Total" and row[3] == "Both Sexes":
            cityLabel = row[4].lower()
            # The if part covers dedectung duplicates:
            if cityLabel in recentCityPopulationUN and recentCityPopulationUN[cityLabel]['year'] <= row[8]:
                if recentCityPopulationUN[cityLabel]['year'] == row[8]:
                    print("Duplicate found!\n" + str(row) + "\n" + str(recentCityPopulationUN[cityLabel]['row']) + "\n")
                # implicit missing 'else' branch: if the year was older than the one already sotred, then ignore this line.
            else:
                recentCityPopulationUN[cityLabel] = {'year': row[8],'pop': float(row[9]), 'row' : row} 

Oops! This produces a lot of duplicates, e.g.:
    
    Duplicate found!
    ['Åland Islands', '2008', 'Total', 'Both Sexes', 'MARIEHAMN', 'City proper', 'Estimate - de jure', 'Final figure, complete', '2009', '10954', '1']
    ['Åland Islands', '2009', 'Total', 'Both Sexes', 'MARIEHAMN', 'City proper', 'Estimate - de jure', 'Final figure, complete', '2009', '11064', '1']
    
What this tells us it that different *reported* years (2nd column) in the data seem to have the same source year (9th column).We can eliminate these duplicates easily by just referring to the reported year instead of the source year:

In [57]:
recentCityPopulationUN={}

with open(undatafile) as f:
    csvfile = csv.reader(f)

    for row in csvfile:
        if len(row) > 10 and row[2] == "Total" and row[3] == "Both Sexes":
            cityLabel = row[4].lower()
            # Note that we changed to row[1] here:
            if cityLabel in recentCityPopulationUN and recentCityPopulationUN[cityLabel]['year'] <= row[1]:
                if recentCityPopulationUN[cityLabel]['year'] == row[1]:
                    print("Duplicate found!\n" + str(row) + "\n" + str(recentCityPopulationUN[cityLabel]['row']) + "\n"  )
            else:
                recentCityPopulationUN[cityLabel] = {'year': row[1],'pop': float(row[9]),'row' : row} 

Ouch! Again there are many duplicates in the data... E.g.:
    
    Duplicate found!
    ['Argentina', '1991', 'Total', 'Both Sexes', 'BUENOS AIRES', 'Urban agglomeration', 'Census - de facto - complete tabulation', 'Final figure, complete', '1991', '11298030', '']
    ['Argentina', '1991', 'Total', 'Both Sexes', 'BUENOS AIRES', 'City proper', 'Census - de facto - complete tabulation', 'Final figure, complete', '1991', '2965403', '']

These seem to indicate that the UN Dataset have also the same city name appearing in different city types: the "Urban agglomeration" area or the "City proper" area.
Let's fix the code to prefer the numbers given for the "Urban agglomeration", in such cases:

In [135]:
recentCityPopulationUN={}

with open(undatafile) as f:
    csvfile = csv.reader(f)

    for row in csvfile:
        if len(row) > 10 and row[2] == "Total" and row[3] == "Both Sexes":
            cityLabel = row[4].lower()
            if (cityLabel in recentCityPopulationUN and recentCityPopulationUN[cityLabel]['year'] <= row[1] and 
                # Adding another condition: only report a duplicate when the new city type is not "City Proper" and the to be replaced type is not "Urban agglomeration" 
                not (row[5] == "Urban agglomeration" and  
                     recentCityPopulationUN[cityLabel]['row'][5] == "City proper")):
                if recentCityPopulationUN[cityLabel]['year'] == row[1]:
                    print("Duplicate found!\n" + str(row) + "\n" + str(recentCityPopulationUN[cityLabel]['row']) + "\n"  )
            else:
                recentCityPopulationUN[cityLabel] = {'year': row[1],'pop': float(row[9]),'row' : row} 

Hmmm, again duplicates, this time of the form:
    
    Duplicate found!
    ['Austria', '2011', 'Total', 'Both Sexes', 'Bregenz', 'City proper', 'Estimate - de jure', 'Final figure, complete', '2011', '27784', '18']
    ['Austria', '2011', 'Total', 'Both Sexes', 'Bregenz', 'City proper', 'Census - de jure - complete tabulation', 'Final figure, complete', '2012', '27831', '']
    
Now these indicate that there are different measure methods... here we could opt to overwrite estimates:

In [124]:
recentCityPopulationUN={}

with open(undatafile) as f:
    csvfile = csv.reader(f)

    for row in csvfile:
        if len(row) > 10 and row[2] == "Total" and row[3] == "Both Sexes":
            cityLabel = row[4].lower()
            if (cityLabel in recentCityPopulationUN and recentCityPopulationUN[cityLabel]['year'] <= row[1] and 
                # Adding another condition: only report a duplicate when the existing city type is not "City Proper" and the to be replaced type is not "Urban agglomeration" 
                not (row[5] == "Urban agglomeration" and  
                     recentCityPopulationUN[cityLabel]['row'][5] == "City proper") and
                # Adding another condition: only report a duplicate when the existing method not is "Estimate ..." and new one is something else. 
                not (row[6][0:7] != "Estimate" and  
                     recentCityPopulationUN[cityLabel]['row'][5][0:7] == "Estimate")):
                if (recentCityPopulationUN[cityLabel]['year'] == row[1] and
                    row[6][0:7] != "Estimate" and  
                     recentCityPopulationUN[cityLabel]['row'][5][0:7] == "Estimate"):
                    print("Duplicate found!\n" + str(row) + "\n" + str(recentCityPopulationUN[cityLabel]['row']) + "\n"  )

            else:
                recentCityPopulationUN[cityLabel] = {'year': row[1],'pop': float(row[9]),'row' : row}

No more duplicates found, although, that was quite ugly!

We could have chosen another resolution strategy, e.g.

* collect all available values for the most recent year and take the average
* collect all available values for the most recent year and take the maximum

... each of those has pros and cons, depending on the use case. 

Let's try one of these strategies with the **merged** UN and EU datasets! But wait a second...

## Did we really resolve all duplicates now?

Have we really resolved all duplicates now?

What about cities with the same names in different countries? Let's check whether such cases exist in the UN dataset!

In [152]:
with open(undatafile) as f:
    csvfile = csv.reader(f)
    
    city_country_dict = {}

    # we want to build up a dictionary to check whether there are cities with the same 
    # name but different countries in our dataset
    # i.e. key = cityname value = country 

    for row in csvfile:
        if len(row) > 10 and row[2] == "Total" and row[3] == "Both Sexes":
            cityLabel = row[4].lower()
            country = row[0].lower()
            if  (cityLabel in city_country_dict and
                 city_country_dict[cityLabel] != country):
                print("Different countries found for City: '" + cityLabel + "': " + country + " and " + city_country_dict[cityLabel]) 
            else:
                city_country_dict[cityLabel] = country 
                

Too bad, e.g.: 
    
    Different countries found for City: london: united kingdom of great britain and northern ireland and canada
    
we can fix this by adding the country code to the city label, remember, we did that before for the Eurostat cities, but here it is more difficult, since we need to lookup the country codes in the ISO file (to arrive at the same lables we had in the EU file!

Let's first build a dictionary of countrycodes... We have seen that last time:

In [89]:
countryCodeFile = './data/iso_3166_2_countries.csv'

countryCodeDict = {}

with open(countryCodeFile) as f:
    csvfile = csv.reader(f)
    for row in csvfile:
        countryCodeDict[row[1]] = row[10]

Now let's use this dictionary to modify the example code from above to rebuild the UN city population dictionary:

In [153]:
recentCityPopulationUN={}

with open(undatafile) as f:
    csvfile = csv.reader(f)

    for row in csvfile:
        if len(row) > 10 and row[2] == "Total" and row[3] == "Both Sexes":
            # This is the only modification: we add a lookup for the countryname to append the 
            # countrycode to the city label: 
            cityLabel = row[4].lower() + countryCodeDict[row[0]] 
            if (cityLabel in recentCityPopulationUN and recentCityPopulationUN[cityLabel]['year'] <= row[1] and 
                not (row[5] == "Urban agglomeration" and  
                     recentCityPopulationUN[cityLabel]['row'][5] == "City proper") and
                not (row[6][0:7] != "Estimate" and  
                     recentCityPopulationUN[cityLabel]['row'][5][0:7] == "Estimate")):
                if (recentCityPopulationUN[cityLabel]['year'] == row[1] and
                    row[6][0:7] != "Estimate" and  
                     recentCityPopulationUN[cityLabel]['row'][5][0:7] == "Estimate"):
                    print("Duplicate found!\n" + str(row) + "\n" + str(recentCityPopulationUN[cityLabel]['row']) + "\n"  )
            else:
                recentCityPopulationUN[cityLabel] = {'year': row[1],'pop': float(row[9]),'row' : row}

KeyError: 'Åland Islands'

Too bad, that doesn't work, since some countries have different names in the countries file from ISO and from the UN.
We would probably need additional data to resolve these... for the moment, we will simply ignore those countries.

In [136]:
recentCityPopulationUN={}

with open(undatafile) as f:
    csvfile = csv.reader(f)

    for row in csvfile:
        if len(row) > 10 and row[2] == "Total" and row[3] == "Both Sexes":
            if row[0] in countryCodeDict:
                cityLabel = row[4].lower() + countryCodeDict[row[0]]
                # print(cityLabel)
            else: continue # continue jumps over this row and continues in the next iteration of the for loop 
            if (cityLabel in recentCityPopulationUN and recentCityPopulationUN[cityLabel]['year'] <= row[1] and 
                # Adding another condition: only report a duplicate when the existing city type is not "City Proper" and the to be replaced type is not "Urban agglomeration" 
                not (row[5] == "Urban agglomeration" and  
                     recentCityPopulationUN[cityLabel]['row'][5] == "City proper") and
                # Adding another condition: only report a duplicate when the existing method not is "Estimate ..." and new one is something else. 
                not (row[6][0:7] != "Estimate" and  
                     recentCityPopulationUN[cityLabel]['row'][5][0:7] == "Estimate")):
                if (recentCityPopulationUN[cityLabel]['year'] == row[1] and
                    row[6][0:7] != "Estimate" and  
                     recentCityPopulationUN[cityLabel]['row'][5][0:7] == "Estimate"):
                    print("Duplicate found!\n" + str(row) + "\n" + str(recentCityPopulationUN[cityLabel]['row']) + "\n"  )
            else:
                recentCityPopulationUN[cityLabel] = {'year': row[1],'pop': float(row[9]),'row' : row}             

Now we should be ready to merge the two datasets! our strategy is to build a joint dictionary `recentCityPopulation`, 
where we collect the average pop `popAvg` `popEU` and `popUN`

In [149]:
recentCityPopulation = {}

for key in recentCityPopulationUN:
    recentCityPopulation[key] = {'popUN': recentCityPopulationUN[key]['pop'],
                                 'pop' : recentCityPopulationUN[key]['pop'] }


for key in recentCityPopulationEU:
    if key in recentCityPopulation:
        recentCityPopulation[key]['popEU'] = recentCityPopulationEU[key]['pop'] 
        recentCityPopulation[key]['pop'] = (recentCityPopulation[key]['pop'] + recentCityPopulation[key]['popEU'])/2 
    else: 
        recentCityPopulation[key] = {'popEU': recentCityPopulationEU[key]['pop'],
                                     'pop' : recentCityPopulationEU[key]['pop'] }
# An example:        
print(recentCityPopulation['wienAT'])

{'popUN': 1829876.0, 'pop': 1785561.0, 'popEU': 1741246}


### Homework: 

How could we resolve the issues with different city and country names in the EU and UN dataset (e.g. "Åland Islands" vs. "Aland", or "Wien" vs. "Vienna", or "Helsinki" vs. "Helsinki / Helsingfors"). Think about strategies or additional datasets you could use! More on that in the last lecture or in the SBWL course 3!